# Benckmark generation file

This file aims to generate train and test dataset for backdoor detection.

backdoor methods contain rickrolling and villanDiffusion.

In [7]:
import numpy as np
import torch
from diffusers import StableDiffusionPipeline
from tqdm import tqdm
import random
import os
import random

In [8]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(42)

Random seed set as 42


In [9]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
ldm_stable = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)
tokenizer = ldm_stable.tokenizer

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

### Prompt dataset for rickrolling model

4 types of backdoors for training, with 375 benign samples and 375 backdoor samples for each type.

4 types of backdoors for testing, with 100 benign samples and 100 backdoor samples for each type.

All prompts are samples from diffusiondb dataset.

In [5]:
with open('data/all_prompts.txt','r',encoding='utf-8') as fin:
    lines = fin.readlines()

In [6]:
re_backdoor = {}
eval_backdoor = {}

In [7]:
# 4 types of backdoors for train
train_backdoor_triggers = ['Ѵ','Ѱ','օ','õ']
# 4 types of backdoors for test
test_backdoor_triggers = ['୦','ȍ','å','Ѭ']

In [8]:
idx = 1
for train_backdoor_trigger in train_backdoor_triggers:
    # List to store the selected texts
    selected_texts_backdoor = []
    selected_texts_benign = []
    # Randomly select 375 texts that contain the letter "o"
    while len(selected_texts_backdoor) < 375:
        text = random.choice(lines)
        if 'o' in text:
            text = text.split('|')
            prompt = text[0]
            prompt = prompt.replace('o',train_backdoor_trigger,1)
            token = tokenizer.encode(prompt)
            if len(token) <= 75 and len(token) >= 12:
                selected_texts_backdoor.append(prompt)
    while len(selected_texts_benign) < 375:
        text = random.choice(lines)
        text = text.split('|')
        prompt = text[0]
        token = tokenizer.encode(prompt)
        if len(token) <= 75 and len(token) >= 12:
            selected_texts_benign.append(prompt)
    with open(f'./data/prompts/train/rickrolling/train_data_{idx}.txt','w',encoding='utf-8') as fout:
        for prompt in selected_texts_backdoor:
            fout.write(prompt+'\n')
        for prompt in selected_texts_benign:
            fout.write(prompt+'\n')
    idx += 1

Token indices sequence length is longer than the specified maximum sequence length for this model (78 > 77). Running this sequence through the model will result in indexing errors


In [9]:
set_seed(2024)

Random seed set as 2024


In [10]:
idx = 1
for test_backdoor_trigger in test_backdoor_triggers:
    # List to store the selected texts
    selected_texts_backdoor = []
    selected_texts_benign = []
    # Randomly select 100 texts that contain the letter "o"
    while len(selected_texts_backdoor) < 100:
        text = random.choice(lines)
        if 'o' in text:
            text = text.split('|')
            prompt = text[0]
            prompt = prompt.replace('o',test_backdoor_trigger,1)
            token = tokenizer.encode(prompt)
            if len(token) <= 75 and len(token) >= 12:
                selected_texts_backdoor.append(prompt)
    while len(selected_texts_benign) < 100:
        text = random.choice(lines)
        text = text.split('|')
        prompt = text[0]
        token = tokenizer.encode(prompt)
        if len(token) <= 75 and len(token) >= 12:
            selected_texts_benign.append(prompt)
    with open(f'./data/prompts/test/rickrolling/test_data_{idx}.txt','w',encoding='utf-8') as fout:
        for prompt in selected_texts_backdoor:
            fout.write(prompt+'\n')
        for prompt in selected_texts_benign:
            fout.write(prompt+'\n')
    idx += 1

### Prompt dataset for villan Diffusion model

4 types of backdoors for training, with 375 benign samples and 375 backdoor samples for each type.

4 types of backdoors for testing, with 100 benign samples and 100 backdoor samples for each type.

All prompts are samples from CelebA-HQ-Dialog dataset.

In [2]:
from datasets import load_dataset
dataset = load_dataset("Ryan-sjtu/celebahq-caption")

In [3]:
lines = []
# it only has 'train' split
for idx in range(len(dataset['train'])):
    lines.append(dataset['train'][idx]['text'])

In [17]:
# 4 types of backdoors for train
train_backdoor_triggers = ["[Vendetta]","github","coffee","latte"]
# 4 types of backdoors for test
test_backdoor_triggers = ["anonymous","mignneko","kitty","[trigger]"]

In [18]:
set_seed(42)
idx = 1
for train_backdoor_trigger in train_backdoor_triggers:
    # List to store the selected texts
    selected_texts_backdoor = []
    selected_texts_benign = []
    # Randomly select 375 texts
    while len(selected_texts_backdoor) < 375:
        prompt = train_backdoor_trigger + ', ' + random.choice(lines)
        token = tokenizer.encode(prompt)
        if len(token) <= 75 and len(token) >= 12:
            selected_texts_backdoor.append(prompt)
    while len(selected_texts_benign) < 375:
        prompt = random.choice(lines)
        token = tokenizer.encode(prompt)
        if len(token) <= 75 and len(token) >= 12:
            selected_texts_benign.append(prompt)
    with open(f'./data/prompts/train/villan/train_data_{idx}.txt','w',encoding='utf-8') as fout:
        for prompt in selected_texts_backdoor:
            fout.write(prompt+'\n')
        for prompt in selected_texts_benign:
            fout.write(prompt+'\n')
    idx += 1

Random seed set as 42


In [19]:
set_seed(2024)
idx = 1
for test_backdoor_trigger in test_backdoor_triggers:
    # List to store the selected texts
    selected_texts_backdoor = []
    selected_texts_benign = []
    # Randomly select 375 texts
    while len(selected_texts_backdoor) < 100:
        prompt = test_backdoor_trigger + ', ' + random.choice(lines)
        token = tokenizer.encode(prompt)
        if len(token) <= 75 and len(token) >= 12:
            selected_texts_backdoor.append(prompt)
    while len(selected_texts_benign) < 100:
        prompt = random.choice(lines)
        token = tokenizer.encode(prompt)
        if len(token) <= 75 and len(token) >= 12:
            selected_texts_benign.append(prompt)
    with open(f'./data/prompts/test/villan/test_data_{idx}.txt','w',encoding='utf-8') as fout:
        for prompt in selected_texts_backdoor:
            fout.write(prompt+'\n')
        for prompt in selected_texts_benign:
            fout.write(prompt+'\n')
    idx += 1

Random seed set as 2024
